Пока жалкие попытки использования `SimpleFeaturizer` и `NlpAlgorithm` классов, не дошла до сколько-нибудь адекватного способа механически по фичам найти возможные антецеденты местоимения (кажется, это равно невыполненной главной задаче). В похожих исследованиях наиболее успешным было использование многослойного перцептрона, RF и SVM, думаю, стоит попробовать обучить RF.
- собираюсь переделать фичи
- надо собрать корпус для обучения, причём похоже, что вручную проставлять связи анафор-антецедент
- (?) добавить алгоритм в класс алгор

In [1]:
from tg.grammar_ru.common import Separator
import pandas as pd

s = 'Это поймёт каждый, если он не глуп. Надо изображать жизнь не такою, как она есть. Слон выронил мяч из хобота, и он покатился.'
db = Separator.build_bundle(s)
db.data_frames['src']

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length
0,0,0,0,0,1,Это,ru,3
1,1,0,1,0,1,поймёт,ru,6
2,2,0,2,0,0,каждый,ru,6
3,3,0,3,0,1,",",punct,1
4,4,0,4,0,1,если,ru,4
5,5,0,5,0,1,он,ru,2
6,6,0,6,0,1,не,ru,2
7,7,0,7,0,0,глуп,ru,4
8,8,0,8,0,1,.,punct,1
9,9,1,0,0,1,Надо,ru,4


In [2]:
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

pmf = PyMorphyFeaturizer()
pmf.featurize(db)
morphology_df = db.data_frames['pymorphy']
morphology_df

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,это,5,0.360000,0.000000,PRCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,понять,1,1.000000,1.000000,VERB,NaN,NaN,sing,NaN,perf,tran,3per,futr,indc,None,None
2,каждый,2,0.621621,0.243243,ADJF,inan,masc,sing,accs,NaN,NaN,NaN,NaN,NaN,None,None
3,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,если,1,1.000000,1.000000,CONJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
5,он,1,1.000000,1.000000,NPRO,NaN,masc,sing,nomn,NaN,NaN,3per,NaN,NaN,None,None
6,не,1,1.000000,1.000000,PRCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
7,глупый,1,1.000000,1.000000,ADJS,NaN,masc,sing,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8,.,1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [3]:
    def _get_pronoun_filter():
        return lambda x: ((x.normal_form == 'он') |
                          (x.normal_form == 'она') |
                          (x.normal_form == 'оно'))

    def _get_row_filter(pronoun_row: pd.Series):
        return lambda x: ((x.gender == pronoun_row.gender) &
                          (x.number == 'sing') &
                          ((x.POS == 'NOUN') |
                           (x.POS == 'ADJF') |
                           (x.POS == 'ADJS') |
                           (x.POS == 'NPRO') |
                           (x.POS == 'PRCL') |
                           (x.POS == 'PRTF') |
                           (x.POS == 'PRTS') |
                           (x.POS == 'ADVB')))

In [4]:
pronouns_df = morphology_df.loc[_get_pronoun_filter(), ['gender']]
pronouns_df['word_id'] = pronouns_df.index
pronouns_df

,gender,word_id
word_id,,
5,masc,5
16,femn,16
26,masc,26


In [5]:
word_ids = db.data_frames['src'][db.data_frames['src']['word_type'] != 'punct'][['word_id']]
merged_df = pd.merge(word_ids,
                             morphology_df[['POS', 'animacy', 'gender', 'number']],
                             on='word_id')
merged_df

,word_id,POS,animacy,gender,number
0,0,PRCL,NaN,NaN,NaN
1,1,VERB,NaN,NaN,sing
2,2,ADJF,inan,masc,sing
3,4,CONJ,NaN,NaN,NaN
4,5,NPRO,NaN,masc,sing
5,6,PRCL,NaN,NaN,NaN
6,7,ADJS,NaN,masc,sing
7,9,PRED,NaN,NaN,NaN
8,10,INFN,NaN,NaN,NaN
9,11,NOUN,inan,femn,sing


In [6]:
left_vicinity = 50

def _get_pronoun_antecedent_candidates(df: pd.DataFrame, row):
    start_index = max(0, row.word_id - left_vicinity)
    preword_df = df[(df['word_id'] >= start_index) & (df['word_id'] < row.word_id)]
    candidates = preword_df.loc[_get_row_filter(row), ['word_id', 'POS', 'animacy']]
    candidates['pronoun_id'] = row.word_id
    return candidates

In [7]:
from yo_fluq_ds import *

antecedent_frames = []
for pronoun_row in Query.df(pronouns_df):
    candidates = _get_pronoun_antecedent_candidates(merged_df, pronoun_row)
    antecedent_frames.append(candidates)
db.data_frames['candidates'] = pd.concat(antecedent_frames)
db.data_frames['candidates']

,word_id,POS,animacy,pronoun_id
2,2,ADJF,inan,5
9,11,NOUN,inan,16
11,13,ADJF,NaN,16
2,2,ADJF,inan,26
4,5,NPRO,NaN,26
6,7,ADJS,NaN,26
15,19,NOUN,anim,26
17,21,NOUN,inan,26
19,23,NOUN,inan,26


In [8]:
from tg.grammar_ru.ml.features import AntecedentFeaturizer

antc = AntecedentFeaturizer()
antc.featurize(db)
db.data_frames['antecedents']

c:\users\alexandra\desktop\grammar_ru\tg\grammar_ru\ml\features\antecedent_featurizer.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_dist.append(index_df.loc[row.word_id:row.pronoun_id - 1]


,word_id,POS,animacy,pronoun_id,candidates_distance,word_distance,sentence_distance,paragraph_distance
0,2,ADJF,inan,5,0,2,0,0
1,11,NOUN,inan,16,1,4,0,0
2,13,ADJF,NaN,16,0,2,0,0
3,2,ADJF,inan,26,5,19,2,0
4,5,NPRO,NaN,26,4,17,2,0
5,7,ADJS,NaN,26,3,15,2,0
6,19,NOUN,anim,26,2,6,0,0
7,21,NOUN,inan,26,1,4,0,0
8,23,NOUN,inan,26,0,2,0,0


In [10]:
pairs = db.data_frames['antecedents'].loc[lambda x: x.sentence_distance < 2, ['word_id', 'pronoun_id']]
pairs

,word_id,pronoun_id
0,2,5
1,11,16
2,13,16
6,19,26
7,21,26
8,23,26


In [11]:
from tg.grammar_ru.algorithms.architecture import NlpAlgorithm

antecedent_counts = pairs['pronoun_id'].value_counts()
db.data_frames['candidates'][NlpAlgorithm.Error] = \
    db.data_frames['candidates']['pronoun_id'].map(lambda x: antecedent_counts[x] > 1)
db.data_frames['candidates']

,word_id,POS,animacy,pronoun_id,error
2,2,ADJF,inan,5,False
9,11,NOUN,inan,16,True
11,13,ADJF,NaN,16,True
2,2,ADJF,inan,26,True
4,5,NPRO,NaN,26,True
6,7,ADJS,NaN,26,True
15,19,NOUN,anim,26,True
17,21,NOUN,inan,26,True
19,23,NOUN,inan,26,True


In [14]:
atc = db.data_frames['antecedents']
df_to_feed_rf = atc.loc[:,(atc.columns != 'word_id') & (atc.columns != 'pronoun_id')]
df_to_feed_rf

,POS,animacy,candidates_distance,word_distance,sentence_distance,paragraph_distance
0,ADJF,inan,0,2,0,0
1,NOUN,inan,1,4,0,0
2,ADJF,NaN,0,2,0,0
3,ADJF,inan,5,19,2,0
4,NPRO,NaN,4,17,2,0
5,ADJS,NaN,3,15,2,0
6,NOUN,anim,2,6,0,0
7,NOUN,inan,1,4,0,0
8,NOUN,inan,0,2,0,0


In [ ]:
# ? from tg.grammar_ru.ml.features import FrequencyFeaturizer